In [5]:
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from edsteva.probes import VisitProbe
from edsteva.utils.file_management import load_object

from ipywidgets import interactive, widgets

import time

from utils import debounce, first_digit_after

sns.set_style("whitegrid")
default_color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
default_linestyle_list = ['-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted']

In [10]:
path = "../results/predictors/visits_synthetic_base.pkl"
visits = VisitProbe()
visits.load(path)

visits.predictor = visits.predictor.drop(columns=[ "stay_source", "c"])

2023-11-23 10:20:01.408 | INFO     | edsteva.utils.file_management:load_object:20 - Successfully loaded from ../results/predictors/visits_synthetic_base.pkl


In [31]:
hospitals = visits.predictor[visits.predictor.care_site_level == "Hôpital"].care_site_short_name.unique().tolist()
ufs = visits.predictor[visits.predictor.care_site_level == "Unité Fonctionnelle (UF)"].care_site_short_name.unique().tolist()

ufs_dict = {
    hospital : [uf for uf in ufs if first_digit_after("-", hospital) == first_digit_after("-",uf)]
    for hospital in hospitals
}

In [33]:
path = "../results/models/fitted_visits.pkl"
rectangle_function_model = load_object(path)
estimates = rectangle_function_model.estimates
estimates.care_site_short_name = estimates.care_site_level + "-" + estimates.care_site_id

2023-11-23 10:26:54.258 | INFO     | edsteva.utils.file_management:load_object:20 - Successfully loaded from ../results/models/fitted_visits.pkl
C:\Users\7046680\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [54]:
output = widgets.Output()

hospital_w = widgets.SelectMultiple(
    options=hospitals,
    value=['Hôpital-1'],
    rows=5,
    description='Hôpital :',
    continuous_update=False
)

stay_type_w = widgets.SelectMultiple(
    options=['urgences', 'hospitalisés', 'consultation',],
    value=["hospitalisés"],
    rows=5,
    description='Stay type :',
)

uf_w = widgets.SelectMultiple(
    options=ufs_dict['Hôpital-1'] + ["no UF"],
    value=["no UF"],
    rows=5,
    description='UF :',
)

@debounce(0.2)
def update(*args):
    with output:
        ufs_hospital = []
        for hospital in hospital_w.value:
            ufs_hospital += ufs_dict[hospital]
        uf_w.options = ufs_hospital + ["no UF"]
        uf_w.value = ["no UF"]
        
hospital_w.observe(update)

def plot_function(hospital, stay_type, ufs):
    
    fig, axs = plt.subplots(ncols=2, figsize=(15, 5))
    predictor = visits.predictor.copy()
    for i, hospit in enumerate(hospital):
        sub_predictor = predictor[predictor.care_site_short_name == hospit]
        for k, t in enumerate(stay_type):
            axs[0].plot(sub_predictor[sub_predictor.stay_type == t].date, 
                     sub_predictor[sub_predictor.stay_type == t].n_visit, 
                     linestyle=default_linestyle_list[k], 
                     c=default_color_list[i % len(default_color_list)], label=hospit+"-"+t)
    axs[0].legend()
    axs[0].set_title("Hôpital")
    
    ufs = [] if "no UF" in ufs else ufs
    for i, uf in enumerate(ufs):
        sub_predictor = predictor[predictor.care_site_short_name == uf]
        for k, t in enumerate(stay_type):
            axs[1].plot(sub_predictor[sub_predictor.stay_type == t].date, 
                     sub_predictor[sub_predictor.stay_type == t].n_visit, 
                     linestyle=default_linestyle_list[k], 
                     c=default_color_list[i % len(default_color_list)], label=uf+"-"+t)
    axs[1].legend() if ufs != [] else None
    axs[1].set_title("UF")

    model = estimates[estimates.stay_type.isin(stay_type) 
                      & (estimates.care_site_short_name.isin(hospital)
                         | estimates.care_site_short_name.isin(ufs)
                         )
                      ].reset_index(drop=True)
    plt.show()

    model.style.set_caption("Care site deployment model")
    display(model)

    
w = interactive(plot_function, hospital=hospital_w, stay_type=stay_type_w, ufs=uf_w, continuous_update=False);

controls = widgets.HBox(w.children[:-1])
output = w.children[-1]
display(widgets.VBox([controls, output]))